In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import tensorboard
import os

# Defining Simple CNN (with Tanh)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 300, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(300, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

# Downloading  and  preparing  CIFAR-10  dataset 

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#images, labels = next(iter(trainloader))

Files already downloaded and verified
Files already downloaded and verified


In [12]:
writer = SummaryWriter()
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        #inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Loss/train', loss.item(), i)
        writer.add_scalar('Accuracy/train', correct/total, i)
       
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            print("Accuracy: ", correct/total)

print('Finished Training')

grid = torchvision.utils.make_grid(inputs)
writer.add_image('images', grid, 0)
writer.add_graph(net, inputs)
writer.close()

[1,  2000] loss: 0.559
Accuracy:  0.80525
[1,  4000] loss: 0.592
Accuracy:  0.7989375
[1,  6000] loss: 0.580
Accuracy:  0.8005833333333333
[1,  8000] loss: 0.601
Accuracy:  0.79690625
[1, 10000] loss: 0.600
Accuracy:  0.795325
[1, 12000] loss: 0.611
Accuracy:  0.7935416666666667
[2,  2000] loss: 0.537
Accuracy:  0.81425
[2,  4000] loss: 0.551
Accuracy:  0.810625
[2,  6000] loss: 0.545
Accuracy:  0.8105416666666667
[2,  8000] loss: 0.571
Accuracy:  0.80796875
[2, 10000] loss: 0.584
Accuracy:  0.80585
[2, 12000] loss: 0.570
Accuracy:  0.8040416666666667
[3,  2000] loss: 0.510
Accuracy:  0.8195
[3,  4000] loss: 0.519
Accuracy:  0.8205
[3,  6000] loss: 0.549
Accuracy:  0.8174583333333333
[3,  8000] loss: 0.521
Accuracy:  0.81815625
[3, 10000] loss: 0.548
Accuracy:  0.8154
[3, 12000] loss: 0.563
Accuracy:  0.8128125
[4,  2000] loss: 0.475
Accuracy:  0.8345
[4,  4000] loss: 0.499
Accuracy:  0.83125
[4,  6000] loss: 0.516
Accuracy:  0.8275
[4,  8000] loss: 0.494
Accuracy:  0.8273125
[4, 10000

# Testing the network on testset and reporting accuracy (with SGD)

In [13]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        #images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 67 %


# Changing optimizer to Adam and running expirement again

In [14]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [15]:
writer = SummaryWriter()
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        #inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Loss/train', loss.item(), i)
        writer.add_scalar('Accuracy/train', correct/total, i)
       
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            print("Accuracy: ", correct/total)

print('Finished Training')

grid = torchvision.utils.make_grid(inputs)
writer.add_image('images', grid, 0)
writer.add_graph(net, inputs)
writer.close()

[1,  2000] loss: 0.317
Accuracy:  0.893375
[1,  4000] loss: 0.325
Accuracy:  0.8905625
[1,  6000] loss: 0.348
Accuracy:  0.8869166666666667
[1,  8000] loss: 0.354
Accuracy:  0.88459375
[1, 10000] loss: 0.354
Accuracy:  0.88305
[1, 12000] loss: 0.363
Accuracy:  0.8806875
[2,  2000] loss: 0.285
Accuracy:  0.90275
[2,  4000] loss: 0.296
Accuracy:  0.9015625
[2,  6000] loss: 0.322
Accuracy:  0.8964583333333334
[2,  8000] loss: 0.333
Accuracy:  0.89315625
[2, 10000] loss: 0.338
Accuracy:  0.890825
[2, 12000] loss: 0.333
Accuracy:  0.8891875
[3,  2000] loss: 0.262
Accuracy:  0.9145
[3,  4000] loss: 0.293
Accuracy:  0.9059375
[3,  6000] loss: 0.296
Accuracy:  0.9025
[3,  8000] loss: 0.309
Accuracy:  0.9005
[3, 10000] loss: 0.318
Accuracy:  0.8977
[3, 12000] loss: 0.325
Accuracy:  0.8957708333333333
[4,  2000] loss: 0.251
Accuracy:  0.917375
[4,  4000] loss: 0.262
Accuracy:  0.9135
[4,  6000] loss: 0.280
Accuracy:  0.91125
[4,  8000] loss: 0.297
Accuracy:  0.9073125
[4, 10000] loss: 0.300
Accu

# Testing the network on testset and reporting accuracy (with Adam)

In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        #images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 65 %
